In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
import numpy as np
import re

In [14]:
zomato_data_path = './datasets/Zomato Bangalore Restaurants Data/zomato_original.csv'
df = pd.read_csv(zomato_data_path)

In [15]:
df.sample(n=10)

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
48171,https://www.zomato.com/bangalore/the-tao-terra...,"5th Floor, 1 MG Mall, Opposite Vivanta by Taj,...",The Tao Terraces,Yes,Yes,3.9 /5,1943,080 49653408,MG Road,Casual Dining,"Salads, Cocktails, Khao Suey, Laksa, Jumbo Pra...","Thai, Japanese, Korean, Burmese, Chinese","2,100","[('Rated 4.0', 'RATED\n Great ambience great ...",[],Dine-out,Residency Road
51258,https://www.zomato.com/bangalore/masala-chai-1...,"G05 Brigade Metropolis, Gorudacharpalya, White...",Masala Chai,Yes,No,4.1 /5,196,080 49545043\n+91 8073696078,Whitefield,"Beverage Shop, Quick Bites","Masala Chai, Sandwiches, Samosa, Masala Tea, I...","Beverages, Fast Food, Tea",250,"[('Rated 5.0', ""RATED\n I have been to masala...","['Masala Maggi', 'Sabudana Khichdi', 'Sabudana...",Dine-out,Whitefield
15797,https://www.zomato.com/bangalore/melange-hotel...,"Hotel Ekaa, 52, Kudlu Gate, Hosur Main Road, N...",Melange - Hotel Ekaa,Yes,Yes,3.2/5,426,080 49343499\r\n+91 9900033245,Electronic City,Casual Dining,"Anjal Masala Fry, Pasta, Veg Platter, Biryani,...","North Indian, Chinese, Continental, Mangalorean",900,"[('Rated 1.0', ""RATED\n We ordered butter chi...",[],Buffet,HSR
22382,https://www.zomato.com/bangalore/quench-the-ju...,"1462, 38th Cross Road, 4th T Block East, Patta...",Quench The Juice Lounge,Yes,No,3.6/5,18,080 43754720,Jayanagar,Beverage Shop,NaN,"Beverages, Juices, Ice Cream",150,"[('Rated 3.0', ""RATED\n Incase you searching ...","['Paneer Sandwich', 'Watermelon Juice [250 ml]...",Delivery,JP Nagar
20804,https://www.zomato.com/bangalore/delhi-food-fa...,"63, 1551, 7th Cross Road, RK Colony, 1st Phase...",Delhi Food Factory,Yes,No,NaN,0,+91 7065797900\r\n+91 9910009994,JP Nagar,Delivery,NaN,North Indian,500,"[('Rated 1.0', ""RATED\n Canceled my firsts or...",[],Delivery,Jayanagar
35852,https://www.zomato.com/bangalore/chaatimes-btm...,"No 04, 1st Main Road, NS Palya, Bannerghatta M...",Chaatimes,Yes,No,4.2 /5,9,+91 8884252255,BTM,Quick Bites,NaN,"Street Food, Fast Food",250,"[('Rated 1.0', 'RATED\n I had ordered puri bh...",[],Delivery,Koramangala 7th Block
5381,https://www.zomato.com/bangalore/sherlocks-caf...,"13/2, Buddha Vihar Road, Frazer Town, Bangalore",Sherlock's Cafe,Yes,No,3.7/5,40,+91 8217021561\r\r\n+91 9986691544,Frazer Town,Cafe,NaN,"Cafe, Fast Food",350,"[('Rated 4.0', 'RATED\n Dropped down to this ...","['Loaded Pizza Fries', 'Loaded Pizza Fries Chi...",Delivery,Brigade Road
49075,https://www.zomato.com/bangalore/mumbai-xpress...,"4th Cross, Bellandur, Annaiah Reddy Layout, Op...",Mumbai Xpress,Yes,No,2.8 /5,92,+91 7760624268,Bellandur,Quick Bites,"Chai, Vada Pav, Pizza","Fast Food, Street Food",350,"[('Rated 1.0', ""RATED\n I can't believe the f...",[],Delivery,Sarjapur Road
13237,https://www.zomato.com/bangalore/1q1-infantry-...,"1, Express Building, Queens Road, Infantry Roa...",1Q1,No,Yes,4.3/5,596,080 48050101,Infantry Road,"Casual Dining, Bar","Cocktails, Stir Fried Noodles, Cheesecake, Jas...","Asian, Japanese, Thai, Malaysian, Vietnamese, ...","2,500","[('Rated 4.0', 'RATED\n Awesome place would r...",[],Drinks & nightlife,Church Street
34019,https://www.zomato.com/bangalore/qube-cafe-kor...,"643, Indiqube Octagon, 80 Feet Road, Koramanga...",Qube Cafe,Yes,No,4.2 /5,170,+91 9535256633,Koramangala 4th Block,Cafe,"Pizza, Coffee, Burgers, Pasta, Tiramisu, Sandw...","Cafe, Desserts",700,"[('Rated 4.0', 'RATED\n The only sad part is ...",[],Dine-out,Koramangala 6th Block


# Check distinct values of output variable

In [16]:
df.rate.value_counts().to_dict()

{'NEW': 2208,
 '3.9/5': 2098,
 '3.8/5': 2022,
 '3.7/5': 2011,
 '3.9 /5': 1874,
 '3.8 /5': 1851,
 '3.7 /5': 1810,
 '3.6/5': 1773,
 '4.0/5': 1609,
 '4.0 /5': 1574,
 '3.6 /5': 1543,
 '4.1 /5': 1474,
 '4.1/5': 1474,
 '3.5/5': 1431,
 '3.5 /5': 1353,
 '3.4/5': 1259,
 '3.4 /5': 1217,
 '3.3/5': 1168,
 '4.2 /5': 1165,
 '3.3 /5': 1142,
 '4.2/5': 1019,
 '3.2/5': 1006,
 '4.3 /5': 917,
 '3.2 /5': 867,
 '3.1/5': 862,
 '4.3/5': 776,
 '3.1 /5': 699,
 '4.4 /5': 628,
 '3.0/5': 558,
 '4.4/5': 519,
 '3.0 /5': 465,
 '2.9/5': 427,
 '4.5 /5': 409,
 '2.9 /5': 375,
 '2.8/5': 313,
 '2.8 /5': 287,
 '4.5/5': 247,
 '4.6 /5': 175,
 '2.7/5': 170,
 '2.6/5': 143,
 '2.7 /5': 137,
 '4.6/5': 125,
 '2.6 /5': 117,
 '4.7 /5': 86,
 '4.7/5': 81,
 '-': 69,
 '2.5 /5': 56,
 '2.5/5': 45,
 '4.8 /5': 43,
 '2.4/5': 40,
 '4.9 /5': 30,
 '2.4 /5': 30,
 '2.3/5': 28,
 '4.9/5': 25,
 '2.3 /5': 23,
 '4.8/5': 23,
 '2.2/5': 19,
 '2.1 /5': 13,
 '2.1/5': 11,
 '2.2 /5': 7,
 '2.0 /5': 7,
 '2.0/5': 4,
 '1.8 /5': 3,
 '1.8/5': 2}

**Observation:** There are many ratings with uncleaned texts we have to manage them

# Feature selection by problem statement
As per our problem statement we will make the feature selection as below for modelling purpose:

| Feature | Comments |
| :- | :- |
| url | Not used URL has nothing to do with ratings |
| address | Raw address is not required as we will focus on location information |
| name | Name has nothing to do with rating |
| online_order | **Required**. We need to model with online_order. Online delivery service plays good role |
| book_table | **Required**. Table booking feature can play good role in ratings |
| rate | **Required**.  Target variable |
| votes | Not selected as new restaurants which will be onboarded will have little or no votes. Cold start problem |
| phone | Not required|
| location | May be required. We are also capturing the info in **listed_in(city)** |
| rest_type | **Required**. Restaurant type might have some effect |
| dish_liked | Not required. New onboarded restaurant will not have this info. |
| cuisines | **Required**. |
| approx_cost(for two people) | **Required**. |
| reviews_list | Not required as of now. Cold start problem. |
| menu_item | **Required**. Requires data cleaning |
| listed_in(type) | **Required**. |
| listed_in(city) | **Required**.  |

In [17]:
df = df.drop(columns=['reviews_list'])
df.sample(10)

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),menu_item,listed_in(type),listed_in(city)
43020,https://www.zomato.com/bangalore/sri-rama-shiv...,"18, Bowring Hospital Road, Shivajinagar, Banga...",Sri Rama,No,No,3.2 /5,4,NaN,Shivajinagar,Quick Bites,NaN,South Indian,200,[],Delivery,MG Road
26333,https://www.zomato.com/bangalore/venkys-xprs-k...,"604,4th Cross, Block 2, Kalyan Nagar, Bangalore",Venky's Xprs,Yes,No,3.7 /5,37,+91 8861286181,Kalyan Nagar,"Quick Bites, Meat Shop",NaN,"Fast Food, Raw Meats",400,[],Dine-out,Kammanahalli
32984,https://www.zomato.com/bangalore/kinpeetos-kor...,"203, KHB Block, Koramangala 5th Block, Bangalore",Kinpeetos,Yes,No,3.9 /5,64,+91 8585982807,Koramangala 5th Block,Quick Bites,"Burgers, Belgian Fries, Pasta, Salads, Falafel...","Lebanese, Greek",300,['Paneer Schezwan with Vegetable Noodles Combo...,Delivery,Koramangala 6th Block
31183,https://www.zomato.com/bangalore/subway-korama...,"2nd Floor, Transit, Forum Mall, Hosur Road, Ko...",Subway,Yes,No,3.7 /5,219,080 22067695,Koramangala 5th Block,"Quick Bites, Food Court","Salads, Chicken Tikka, Veg Shammi, Teriyaki Ch...","Salad, Fast Food",600,[],Dine-out,Koramangala 5th Block
22065,https://www.zomato.com/bangalore/masale-daan-j...,"1260 SY 35/4 SJR Tower's, 7th Phase, 24th Main...",Masale Daan,Yes,No,NaN,0,+91 8043334333,JP Nagar,"Takeaway, Delivery",NaN,North Indian,350,['Jeera Rice + Lal Moth + Paneer Tikka Masala'...,Delivery,JP Nagar
49524,https://www.zomato.com/bangalore/flavored-with...,"A-112, SLS Splendor Apartments, Devrabisanhall...",Flavored With Love,No,No,NEW,0,+91 9897006443,Bellandur,"Takeaway, Delivery",NaN,Bakery,200,[],Delivery,Sarjapur Road
2805,https://www.zomato.com/bangalore/a2b-veg-adyar...,"200, Next to Canara Bank, KG Road, Gandhi Baza...",A2B Veg - Adyar Ananda Bhavan,Yes,No,2.7/5,97,080 26621222\r\n080 41203999,Basavanagudi,Casual Dining,Masala Dosa,"South Indian, Fast Food, Street Food",400,[],Delivery,Basavanagudi
44372,https://www.zomato.com/bangalore/kolkata-famou...,"Ramanna Gardens, Chamundeswari Layout, Jalahal...",Kolkata Famous Kati Rolls,No,No,NaN,0,080 33512882,Jalahalli,Quick Bites,NaN,Rolls,150,[],Delivery,New BEL Road
3686,https://www.zomato.com/bangalore/mm-tiffen-cen...,"29, 4th Main Rd, Srinivasnagar, Banashankari, ...",MM Tiffen Center,No,No,NaN,0,+91 9164551923,Banashankari,Quick Bites,NaN,"South Indian, Fast Food",150,[],Dine-out,Basavanagudi
20537,https://www.zomato.com/bangalore/tasty-bites-1...,"8, 23rd Main Marenahalli, 2nd Phase, JP Nagar,...",Tasty Bites,Yes,No,3.8/5,217,+91 9980621655\r\n+91 9845342085,JP Nagar,Quick Bites,"Paratha, Kheer, Lassi, Buttermilk, Paneer Bhur...",North Indian,450,"['Chole', 'Dal Tadka', 'Dal Khichdi', 'Gobi Pa...",Delivery,Jayanagar


# Column Renames

In [18]:
rename_dict = {'approx_cost(for two people)':'cost'
               ,'listed_in(type)':'listed_in_type'
               ,'listed_in(city)':'listed_in_city'
              }
df = df.rename(columns=rename_dict)
df.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,cost,menu_item,listed_in_type,listed_in_city
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,[],Buffet,Banashankari


**Observation:**
1. cuisines, menu_item, rest_type require count vectorization using ', ' separated values
1. online_order, book_table, listed_in_city, listed_in_type require one hot encoding

# Simple EDA

## Categorical columns

In [19]:
categorical_cols = ['online_order',
                    'book_table',
                    'location',
                    'rest_type',
                    'listed_in_type',
                    'listed_in_city'
                   ]
for col in categorical_cols:
    print('****{}****'.format(col))
    print(df[col].value_counts().to_dict())
    print('-'*5,'NaNs present','-'*5)
    print(df[col].isna().sum())

****online_order****
{'Yes': 30444, 'No': 21273}
----- NaNs present -----
0
****book_table****
{'No': 45268, 'Yes': 6449}
----- NaNs present -----
0
****location****
{'BTM': 5124, 'HSR': 2523, 'Koramangala 5th Block': 2504, 'JP Nagar': 2235, 'Whitefield': 2144, 'Indiranagar': 2083, 'Jayanagar': 1926, 'Marathahalli': 1846, 'Bannerghatta Road': 1630, 'Bellandur': 1286, 'Electronic City': 1258, 'Koramangala 1st Block': 1238, 'Brigade Road': 1218, 'Koramangala 7th Block': 1181, 'Koramangala 6th Block': 1156, 'Sarjapur Road': 1065, 'Ulsoor': 1023, 'Koramangala 4th Block': 1017, 'MG Road': 918, 'Banashankari': 906, 'Kalyan Nagar': 853, 'Richmond Road': 812, 'Frazer Town': 727, 'Malleshwaram': 725, 'Basavanagudi': 684, 'Residency Road': 675, 'Banaswadi': 664, 'Brookefield': 658, 'New BEL Road': 649, 'Kammanahalli': 648, 'Rajajinagar': 591, 'Church Street': 569, 'Lavelle Road': 529, 'Shanti Nagar': 511, 'Shivajinagar': 499, 'Domlur': 496, 'Cunningham Road': 491, 'Old Airport Road': 446, 'Ejipu

**Observation:** 
1. We can substitute some locations which have very less counts as "Others"
1. rest_type is not a one-to-one mapping. It needs to be handled through count Vectorizer.
1. There can be direct correlation of **listed_in(city)** and **location**. As per our understanding **listed_in(city)** seems to be more drill down of **location**. So for overall view, we can go with **listed_in(city)**. Need to confirm with B'lore people on these. However there are presence of NaNs in the **location** field. So we will use **listed_in(city)** for it has complete information.
1. Fields like **rest_type** contains too many categories which are comma-separated. We need to perform encoding by count vectorization

## Preprocess the columns
Since rating column is given in the form of string we need to preprocess it to float

In [20]:
def preprocess_rating(rating):
    try:
        rate = float(rating.split('/')[0].strip())
    except:
        rate = None
    return rate
def preprocess_cost(cost):
    try:
        cost_cleaned = float(re.sub('\,','',cost))
    except:
        cost_cleaned = None
    return cost_cleaned
def preprocess_menu_item(menu_item:list):
    try:
        menu = ', '.join(eval(menu_item))
    except:
        menu = None
    return menu
df['rate'] = df.rate.apply(lambda element: preprocess_rating(element))
df['cost'] = df['cost'].apply(lambda element: preprocess_cost(element))
df['menu_item'] = df['menu_item'].apply(lambda element: preprocess_menu_item(element))

**Observation:** So our custom column cleaner is working fine we can drop these original columns

In [21]:
df.sample(10).dtypes

url                object
address            object
name               object
online_order       object
book_table         object
rate              float64
votes               int64
phone              object
location           object
rest_type          object
dish_liked         object
cuisines           object
cost              float64
menu_item          object
listed_in_type     object
listed_in_city     object
dtype: object

# Segregate the unrated restaurants

In [22]:
unrated_rest = df[df.rate.isna()]
df = df[~df.rate.isna()]

In [23]:
display('Unrated restaurants:::',unrated_rest.sample(10))
display('Rated restaurants:::',df.sample(10))

'Unrated restaurants:::'

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,cost,menu_item,listed_in_type,listed_in_city
14176,https://www.zomato.com/bangalore/royal-south-r...,"4, S.R.T Alphines Luxurious Hotel, Narayana Hr...",Royal South Restaurant,No,No,NaN,0,080 27831414\r\n080 27831515,Electronic City,Casual Dining,NaN,"South Indian, Chinese",650.0,,Dine-out,Electronic City
17471,https://www.zomato.com/bangalore/five-satr-chi...,"Green Glen Layout, Bellandur, Bengalore",Five Satr Chiken,No,No,NaN,0,+91 9742975189\r\n+91 9148642547,Bellandur,Quick Bites,NaN,Fast Food,300.0,,Dine-out,HSR
1609,https://www.zomato.com/bangalore/sri-annalaksh...,"Begur Koppa Main Road, Bannerghatta Road, Bang...",Sri Annalakshmi,No,No,NaN,0,+91 9036008000\r\n+91 8050017100,Bannerghatta Road,"Takeaway, Delivery",NaN,"South Indian, North Indian",200.0,,Delivery,Bannerghatta Road
854,https://www.zomato.com/bangalore/food-plaza-ba...,"23, Basavanagudi, Bangalore",Food Plaza,No,No,NaN,0,+91 9945404466,Basavanagudi,Quick Bites,NaN,"South Indian, North Indian, Chinese",150.0,,Dine-out,Banashankari
31580,https://www.zomato.com/bangalore/tanu-dis-kitc...,"36/1, Sarjapur Main Road, Jakkasandra, 1st Blo...",Tanu Di's Kitchen,Yes,No,NaN,0,+91 9108608305,Koramangala 1st Block,Quick Bites,NaN,"North Indian, South Indian, Bengali",350.0,,Dine-out,Koramangala 5th Block
1306,https://www.zomato.com/bangalore/lassi-shop-jp...,"24th Main, JP Nagar, Bangalore",Lassi Shop,Yes,No,NaN,0,+91 8105692341,JP Nagar,Dessert Parlor,NaN,"Ice Cream, Beverages",80.0,,Delivery,Bannerghatta Road
12845,https://www.zomato.com/bangalore/gowdara-naati...,"50/1, Meenal Tower, 1st Cross, Residency Road,...",Gowdara Naati Mane,No,No,NaN,0,+91 9945101234\r\n+91 9741110999,Residency Road,Casual Dining,NaN,"South Indian, Biryani",600.0,,Dine-out,Church Street
12849,https://www.zomato.com/bangalore/zuiqa-taste-r...,"Near Errapa Garden, Old SBI Building, Near BDF...",Zuiqa Taste,No,No,NaN,0,+91 8025309959\r\n+91 7975646080,Richmond Road,Quick Bites,NaN,"North Indian, Chinese",300.0,,Dine-out,Church Street
13189,https://www.zomato.com/bangalore/arhams-resto-...,"6/3,8th Main Road, Vasanth Nagar, Bangalore",Arham's Resto,No,No,NaN,0,080 22262224,Vasanth Nagar,Quick Bites,NaN,"Chinese, North Indian, Fast Food",250.0,,Dine-out,Church Street
6982,https://www.zomato.com/bangalore/momos-me-broo...,"Near Apollo Clinic, Kundalahalli Gate, Thubara...",Momos & Me,Yes,No,NaN,0,+91 8971954197\r\n+91 9739604557,Brookefield,"Takeaway, Delivery",NaN,Chinese,200.0,"Veg Manchow Soup, Veg Sweet Corn Soup, Chicken...",Delivery,Brookefield


'Rated restaurants:::'

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,cost,menu_item,listed_in_type,listed_in_city
39226,https://www.zomato.com/bangalore/calcutta-chat...,"25/1 A, Millers Road, Vasanth Nagar, Bangalore",Calcutta Chats,No,No,3.6,14,+91 8892697088,Vasanth Nagar,Quick Bites,NaN,North Indian,400.0,,Dine-out,Lavelle Road
49549,https://www.zomato.com/bangalore/waffle-talesz...,"65/1A, Sarjapur Main Road, Kaikondrahalli, Sar...",Waffle Talesz,Yes,No,3.8,33,080 49652850,Sarjapur Road,Dessert Parlor,NaN,Desserts,200.0,"Belgian Waffle with Nutella [2 Pieces], Belgia...",Desserts,Sarjapur Road
4399,https://www.zomato.com/bangalore/punjabi-chull...,"301/A, 3rd B Cross, Kariyappa Building, Behind...",Punjabi Chulla,Yes,No,3.2,9,+91 9916824979,Bellandur,Delivery,NaN,North Indian,300.0,,Delivery,Bellandur
32488,https://www.zomato.com/bangalore/aliensip-1-bt...,"24, Shop 1, Ground Floor, 3rd Cross, 18th Main...",AlienSip,Yes,No,3.6,11,+91 9916641504\r\n+91 9886839331,BTM,Beverage Shop,NaN,Juices,200.0,,Delivery,Koramangala 6th Block
36625,https://www.zomato.com/bangalore/oms-cafe-pran...,"18, 36, Ground Floor, Raheja Arcade, 7th Block...",Om's Cafe Prana,No,No,3.6,17,+91 9844708137,Koramangala 7th Block,Cafe,NaN,"Cafe, Fast Food",300.0,,Dine-out,Koramangala 7th Block
13932,https://www.zomato.com/bangalore/rags-on-fire-...,"5th Floor, Uniworld Commercial Complex, Phase ...",Rags on Fire,Yes,No,3.1,64,+91 9108174120\r\n+91 9599739603,Electronic City,Cafe,"Soya Chaap, Biryani",Cafe,600.0,,Delivery,Electronic City
23382,https://www.zomato.com/bangalore/cafe-bollywoo...,"4 M Block, Central Mall, 9th Block, JP Nagar, ...",Cafe Bollywood,No,No,3.4,25,+91 9833440640,JP Nagar,Quick Bites,"Pav Bhaji, Dahipuri, Chaat, Paratha, Noodles",Street Food,200.0,,Dine-out,JP Nagar
31716,https://www.zomato.com/bangalore/royal-view-re...,"51/1, Part 3, Hosur Road, Beside Market Square...",Royal View Restaurant,Yes,No,3.7,26,080 41133114\r\n080 41233571,BTM,Casual Dining,NaN,"North Indian, Chinese, Arabian",700.0,,Dine-out,Koramangala 5th Block
10498,https://www.zomato.com/bangalore/melt-caf%C3%A...,"1st Main, Bhuvanappa Layout, BTM, Bangalore",Melt,No,No,4.0,115,+91 8041656568\r\r\r\n+91 8095495497,BTM,Casual Dining,"Burgers, Nachos, Fries, White Sauce Pasta, Che...","Continental, Steak, Fast Food",1000.0,,Dine-out,BTM
38302,https://www.zomato.com/bangalore/donne-biriyan...,"17, 5th Main Road, Opposite Moti Mahal, Gandhi...",Donne Biriyani Mane,No,No,3.2,6,+91 7411600800,Majestic,Quick Bites,NaN,Biryani,300.0,,Delivery,Lavelle Road


# Save the DF

In [24]:
df.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,cost,menu_item,listed_in_type,listed_in_city
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800.0,,Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800.0,,Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800.0,,Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300.0,,Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600.0,,Buffet,Banashankari


In [25]:
df.to_csv('./datasets/Zomato Bangalore Restaurants Data/zomato.csv', index=False)

In [26]:
df.isna().sum()

url                   0
address               0
name                  0
online_order          0
book_table            0
rate                  0
votes                 0
phone               814
location              0
rest_type           149
dish_liked        18203
cuisines             11
cost                247
menu_item             0
listed_in_type        0
listed_in_city        0
dtype: int64

In [16]:
df.sample(10)

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,cost,reviews_list,menu_item,listed_in_type,listed_in_city
17430,https://www.zomato.com/bangalore/a-m-new-punja...,"49/6-15, 3rd Main Road, Janatha Colony, Jakkas...",A.M New Punjab Food Corner,Yes,No,3.5,21,+91 9900604263,Koramangala 1st Block,Quick Bites,NaN,"North Indian, Chinese",350.0,"[('Rated 1.0', ""RATED\n Ordered chicken fried...",,Dine-out,HSR
31866,https://www.zomato.com/bangalore/bruncherz-btm...,"1st Main, S.G Palya, Christ Pool Main Road",Bruncherz,No,No,3.5,15,+91 9481160720\r\n+91 9900690335,BTM,Quick Bites,NaN,"North Indian, Chinese",200.0,"[('Rated 4.0', 'RATED\n The place is good if ...",,Dine-out,Koramangala 5th Block
37826,https://www.zomato.com/bangalore/donne-biryani...,"45, Opposite Mayo Hall, Residency Road, Bangalore",Donne Biryani House,Yes,No,3.4,70,+91 9663783088,Residency Road,Quick Bites,NaN,Biryani,400.0,"[('Rated 4.0', 'RATED\n Trending right now fo...",,Delivery,Lavelle Road
44219,https://www.zomato.com/bangalore/absolute-waff...,"7, Inside MSR Campus, New BEL Road, Bangalore",Absolute Waffles,Yes,No,3.9,49,+91 9243494002,New BEL Road,Dessert Parlor,Waffles,"Desserts, Fast Food",200.0,"[('Rated 5.0', 'RATED\n We ordered Ferrero bl...","Ferrero Blast Waffle, Lady in Red Waffle, Abso...",Delivery,New BEL Road
14871,https://www.zomato.com/bangalore/asia-in-a-box...,"21, 2nd Cross, 3rd Main, Kirthi Layout, Kachar...",Asia In A Box,Yes,No,3.0,16,080 41606452\r\n080 41606454,Kammanahalli,Casual Dining,NaN,"Thai, Vietnamese, Indonesian, Asian",550.0,"[('Rated 1.0', 'RATED\n The beef rendang was ...",,Delivery,Frazer Town
25910,https://www.zomato.com/bangalore/ammas-pastrie...,"111/B, Ground Floor, 1st Main, 1st Stage, 1st ...",Amma's Pastries,No,No,3.9,130,+91 9590607750,HBR Layout,Bakery,"Veg Puff, Egg Puff",Bakery,400.0,"[('Rated 5.0', 'RATED\n I know the owners and...",,Desserts,Kammanahalli
49654,https://www.zomato.com/bangalore/mughal-treat-...,"334, Doddkanhalli, Near WIPRO Corporate Office...",Mughal Treat,Yes,No,3.6,116,080 49652673,Sarjapur Road,Casual Dining,"Charcoal Chicken, Shawarma Roll, Chicken Curry...","Mughlai, North Indian, Chinese",700.0,"[('Rated 5.0', 'RATED\n Oh Man! The Chicken J...","Charcoal Chicken, Afghani Chicken, Andhra Styl...",Dine-out,Sarjapur Road
37189,https://www.zomato.com/bangalore/tree-top-wils...,"Temple Tree Hotel, 3, 9th Cross, Wilson Garden...",Tree Top,Yes,No,3.8,75,080 49652744,Wilson Garden,Casual Dining,Friendly Staff,"Continental, Chinese, North Indian, South Indian",1000.0,"[('Rated 3.0', 'RATED\n I was visited this re...","Murgh Ka Soola, Veg Biryani, Chicken Biryani, ...",Dine-out,Koramangala 7th Block
11707,https://www.zomato.com/bangalore/popat-indiran...,"73/1, Shamanna Gowda Layout, 1st Main Channel ...",Popat,Yes,No,3.2,41,+91 7015836528,Indiranagar,Quick Bites,NaN,North Indian,300.0,"[('Rated 5.0', ""RATED\n I found this restaura...","Jeera Rice, Pulao Rice, Matar Pulao, Veg Birya...",Delivery,Church Street
37751,https://www.zomato.com/bangalore/the-courtyard...,"105, Double Road, Shanti Nagar, Bangalore",The Courtyard,No,Yes,3.9,300,+91 7338677996,Shanti Nagar,Cafe,"Hot Chocolate, Sandwiches, Garden Pizza, Pasta...","Cafe, Italian",750.0,"[('Rated 3.0', 'RATED\n This cafe is situated...",,Cafes,Lavelle Road
